<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li></ul></div>

# Introduction

In [1]:
from splinter import Browser
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from datetime import datetime
from sys import getsizeof
import re

In [6]:
def surfline_scraper(url):

    #init browser
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    scraped_datetime = datetime.now()
    time_update_str = soup.find('div', class_="quiver-forecaster-profile__update-container").find('span').contents[3]
    time_update = datetime.strptime(re.findall(r'\d+:\d+[pa]m', time_update_str)[0], '%I:%M%p')

    condition = soup.find('div', class_='sl-spot-report').contents[0].text

    surf_height_range = list(map(float, re.findall(r'\d+', soup.find('span', class_='quiver-surf-height').contents[0])))
    surf_height = sum(surf_height_range) / len(surf_height_range)

    swell_one = list(map(float, re.findall(r'\d+\.\d+|\d+', soup.find('div', class_='sl-condition-stats__swells').contents[0].text)))
    swell_one_ft = swell_one[0]
    swell_one_secs = swell_one[1]
    swell_one_degrees = swell_one[2]

    swell_two = list(map(float, re.findall(r'\d+\.\d+|\d+', soup.find('div', class_='sl-condition-stats__swells').contents[1].text)))
    swell_two_ft = swell_two[0]
    swell_two_secs = swell_two[1]
    swell_two_degrees = swell_two[2]

    swell_three = list(map(float, re.findall(r'\d+\.\d+|\d+', soup.find('div', class_='sl-condition-stats__swells').contents[2].text)))
    swell_three_ft = swell_three[0]
    swell_three_secs = swell_three[1]
    swell_three_degrees = swell_three[2]               

    wind_mph = float(re.findall(r'\d+', soup.find_all('span', class_='sl-reading')[1].text)[0])
    wind_degrees = float(re.findall(r'\d+', soup.find_all('span', class_='sl-reading-description')[2].text)[0])

    tide_current = float(soup.find('span', class_='sl-reading').contents[0])
    tide_local_extrema = soup.find_all('span', class_='sl-reading-description')[1].text
    tide_local_extrema_ft = float(re.findall(r'-?\d+\.\d+', tide_local_extrema)[0])
    tide_local_extrema_time = datetime.strptime(re.findall(r'\d+:\d+[pa]m', tide_local_extrema)[0], '%I:%M%p')

    weather = float(soup.find('div', class_='sl-wetsuit-recommender__conditions__weather').find('div').contents[1])
    water_temp_range = soup.find('div', class_='sl-wetsuit-recommender__conditions__water').find('div')
    water_temp = (float(water_temp_range.contents[1]) + int(water_temp_range.contents[5])) / 2

    day_light_attr = soup.find('table', class_='sl-forecast-graphs__table sl-forecast-graphs__table--sunlight-times').find_all('td')
    first_light = datetime.strptime(day_light_attr[1].text, '%I:%M%p')
    sunrise = datetime.strptime(day_light_attr[3].text, '%I:%M%p')
    sunset = datetime.strptime(day_light_attr[5].text, '%I:%M%p')
    last_light = datetime.strptime(day_light_attr[7].text, '%I:%M%p')

    description = ' '.join(p.text for p in soup.find('div', class_='sl-spot-report').find_all('p')).replace('\n', '').replace('\xa0', '')

    #init cursor location on chart header above surf cam video
    element = driver.find_elements_by_xpath('//div[@class="sl-forecast-header__nav__page-level__link__text"]')[2]
    action = webdriver.common.action_chains.ActionChains(driver)

    #move cursor to surf cam block
    action.move_to_element_with_offset(element, 5, 200)

    #start surf cam video
    action.click()
    action.perform()
    

    ad_button = soup.find('div', class_="jw-icon jw-icon-inline jw-button-color jw-reset jw-icon-playback")['aria-label']

    #Check for when the advertisement stops
    while ad_button != 'Stop':
        time.sleep(10)
        ad_button = BeautifulSoup(driver.page_source, 'html.parser').find('div', class_="jw-icon jw-icon-inline jw-button-color jw-reset jw-icon-playback")['aria-label']
        print(ad_button)
    
    #full screen surf cam
    action.double_click()
    action.perform()
    ad_button = BeautifulSoup(driver.page_source, 'html.parser').find('div', class_="jw-icon jw-icon-inline jw-button-color jw-reset jw-icon-playback")['aria-label']
    
    #Check for when the advertisement 2 stops
    while ad_button != 'Stop':
        time.sleep(10)
        ad_button = BeautifulSoup(driver.page_source, 'html.parser').find('div', class_="jw-icon jw-icon-inline jw-button-color jw-reset jw-icon-playback")['aria-label']
        print(ad_button)
    time.sleep(4)
    
    #save surf cam screen shot
    driver.save_screenshot('{}{}{}_{}_{}.png'.format(scraped_datetime.year, scraped_datetime.month, 
                                                     scraped_datetime.day, scraped_datetime.hour, 
                                                     scraped_datetime.minute))

    driver.close()
    driver.quit()

    surfline_dict = {
        'scraped_time': scraped_datetime,
        'time_updated': time_update,
        'surf_height': surf_height,
        'swell_one_ft': swell_one_ft,
        'swell_one_secs': swell_one_secs,
        'swell_one_degrees': swell_one_degrees,
        'swell_two_ft': swell_two_ft,
        'swell_two_secs': swell_two_secs,
        'swell_two_degrees': swell_two_degrees,
        'swell_three_ft': swell_three_ft,
        'swell_three_secs': swell_three_secs,
        'swell_three_degrees': swell_three_degrees,
        'tide_current': tide_current,
        'tide_local_extrema': tide_local_extrema_ft,
        'tide_local_extrema_time': tide_local_extrema_time,
        'wind_mph': wind_mph,
        'wind_degrees': wind_degrees,
        'condition': condition,
        'weather': weather,
        'water_temp': water_temp,
        'first_light': first_light,
        'sunrise': sunrise,
        'sunset': sunset,
        'last_light': last_light,
        'description': description
        }

In [7]:
url = 'https://www.surfline.com/surf-report/malibu-overview/5c9134bb8e8d940001706997'
surfline_scraper(url)

Pause
Pause
Pause
Stop
Play
Pause
Stop
